In [34]:
import os
import sys
import pickle 
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor, GradientBoostingRegressor 
from sklearn.metrics import mean_pinball_loss, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

import src.data.preprocessor as pre
import src.data.datasets as data
import src.helper as h

In [55]:
dataset_name = "CMAPSS2"
cal_portion=0.1
exp_seed = 0

In [56]:
removable_cols = ["sm01", "sm05", "sm06", "sm10", "sm16", "sm18", "sm19"]
ignore_columns = ["time", "os1", "os2", "os3"]

alpha_array = np.array([0.1, 0.15, 0.2, 0.25]) #array of miscoverage rates
quantiles = np.concatenate((alpha_array, np.array([0.5]), 1-alpha_array))

#load, split, and preprocess the specified dataset 
dataset = data.get_dataset(dataset_name, MinMaxScaler(feature_range=(-1, 1)))
split_dataset = pre.split_dataset(dataset, cal_size=cal_portion, random_state=exp_seed)
proc_dataset = pre.preprocess_split(split_dataset, scaler_factory=dataset["scaler_factory"], window_size=1, removable_cols=removable_cols, ignore_columns=ignore_columns)

X_train = proc_dataset["train"]["X"].reshape((-1,14))
y_train = proc_dataset["train"]["y"].reshape(-1)
X_cal = proc_dataset["cal"]["X"].reshape((-1,14))
y_cal = proc_dataset["cal"]["y"].reshape(-1)
idx_cal = proc_dataset["cal"]["index"]
X_test, y_test, idx_test = h.reform_test_data(proc_dataset["test"])
X_test = X_test.reshape((-1,14))
y_test = y_test.reshape(-1)


In [57]:
all_models = {}
for q in quantiles:
    gbr = HistGradientBoostingRegressor(loss="quantile", quantile=q, random_state=exp_seed)
    all_models["q %1.2f" % q] = gbr.fit(X_train, y_train)
    print("training mean absolute error of GB:", mean_absolute_error(y_train, gbr.predict(X_train)))
    print("calibration mean absolute error of GB:", mean_absolute_error(y_cal, gbr.predict(X_cal)))

training mean absolute error of GB: 26.66881209754944
calibration mean absolute error of GB: 29.5310719297658
training mean absolute error of GB: 22.349861259424983
calibration mean absolute error of GB: 24.743663171900366
training mean absolute error of GB: 19.592216434914356
calibration mean absolute error of GB: 21.438151258104995
training mean absolute error of GB: 17.42208411100241
calibration mean absolute error of GB: 18.72778628970242
training mean absolute error of GB: 13.565645241904358
calibration mean absolute error of GB: 13.297863307995026
training mean absolute error of GB: 29.711999274825533
calibration mean absolute error of GB: 31.412831649806538
training mean absolute error of GB: 20.478672458703876
calibration mean absolute error of GB: 20.8817155871189
training mean absolute error of GB: 19.064353526926233
calibration mean absolute error of GB: 19.56339388119991
training mean absolute error of GB: 16.872551769465304
calibration mean absolute error of GB: 16.9933023

In [35]:
all_models = {}
for q in quantiles:
    gbr = GradientBoostingRegressor(loss="quantile", alpha=q, random_state=exp_seed, n_estimators=500)
    all_models["q %1.2f" % q] = gbr.fit(X_train, y_train)
    print("training mean absolute error of GB:", mean_absolute_error(y_train, gbr.predict(X_train)))
    print("calibration mean absolute error of GB:", mean_absolute_error(y_cal, gbr.predict(X_cal)))

training mean absolute error of GB: 25.305121821554977
calibration mean absolute error of GB: 32.96799273109887
training mean absolute error of GB: 21.70427996198666
calibration mean absolute error of GB: 28.574233921020188
training mean absolute error of GB: 18.648215630889556
calibration mean absolute error of GB: 24.534115382544577
training mean absolute error of GB: 16.241115179313173
calibration mean absolute error of GB: 20.99808849310661
training mean absolute error of GB: 12.08646541923864
calibration mean absolute error of GB: 11.965019451256707
training mean absolute error of GB: 28.11748078478653
calibration mean absolute error of GB: 24.267049119346655
training mean absolute error of GB: 38.51064985872637
calibration mean absolute error of GB: 35.36147283340817
training mean absolute error of GB: 17.022445729052915
calibration mean absolute error of GB: 14.258710744088434
training mean absolute error of GB: 38.51064985872637
calibration mean absolute error of GB: 35.3614728

In [58]:
gbr_ls = HistGradientBoostingRegressor(loss="squared_error", random_state=exp_seed)
all_models["mse"] = gbr_ls.fit(X_train, y_train)
print("training mean absolute error of GB:", mean_absolute_error(y_train, gbr_ls.predict(X_train)))
print("calibration mean absolute error of GB:", mean_absolute_error(y_cal, gbr_ls.predict(X_cal)))

training mean absolute error of GB: 13.99311732443111
calibration mean absolute error of GB: 14.035084814355077


In [68]:
gbr_ls = GradientBoostingRegressor(loss="squared_error", random_state=exp_seed)
all_models["mse"] = gbr_ls.fit(X_train, y_train)
print("training mean absolute error of GB:", mean_absolute_error(y_train, gbr_ls.predict(X_train)))
print("calibration mean absolute error of GB:", mean_absolute_error(y_test, gbr_ls.predict(X_test)))

training mean absolute error of GB: 15.889087368742244
calibration mean absolute error of GB: 14.49726354484342


In [69]:
gbr_ls = GradientBoostingRegressor(loss="squared_error", random_state=exp_seed)
all_models["mse"] = gbr_ls.fit(X_train, y_train)
print("training mean absolute error of GB:", np.sqrt(mean_squared_error(y_train, gbr_ls.predict(X_train))))
print("calibration mean absolute error of GB:", np.sqrt(mean_squared_error(y_test, gbr_ls.predict(X_test))))

training mean absolute error of GB: 20.569370613427434
calibration mean absolute error of GB: 18.49255816955265


In [59]:
y_hat_train = gbr_ls.predict(X_train)
y_hat_cal = gbr_ls.predict(X_cal)
y_hat_test = gbr_ls.predict(X_test)
res_train = np.abs(y_hat_train - y_train) 
res_cal = np.abs(y_hat_cal - y_cal) 

In [60]:
RF = RandomForestRegressor(random_state=exp_seed) 
RF.fit(X_train, res_train)
print("Random forest details:")
print("average labels (training absolute residuals of DCNN):", res_train.mean())
print("training mean absolute error of RF:", mean_absolute_error(res_train, RF.predict(X_train)))
print("calibration mean absolute error of RF:", mean_absolute_error(res_cal, RF.predict(X_cal)))

Random forest details:
average labels (training absolute residuals of DCNN): 13.99311732443111
training mean absolute error of RF: 2.7705612547223555
calibration mean absolute error of RF: 7.497478684194068


In [61]:
sigma_cal = RF.predict(X_cal)
sigma_test = RF.predict(X_test)

In [62]:
y_hat_cal_CQR ={}
y_hat_test_CQR ={}
for q in quantiles:
    y_hat_cal_CQR["q %1.2f" % q] = all_models["q %1.2f" % q].predict(X_cal)
    y_hat_test_CQR["q %1.2f" % q] = all_models["q %1.2f" % q].predict(X_test)

In [63]:
scores = np.abs(y_cal - y_hat_cal) 
scores_normalized = scores/sigma_cal

In [64]:
rho = 0.99
for i in range(len(alpha_array)):
    alpha = alpha_array[i]
    q = h.compute_quantile(scores, alpha)
    q_array = h.compute_quantiles_nex(rho, scores.reshape((-1,1)), idx_test, idx_cal, alpha).reshape(-1)
    q_normalized = h.compute_quantile(scores_normalized, alpha)
    q_array_normalized = h.compute_quantiles_nex(rho, scores_normalized.reshape((-1,1)), idx_test, idx_cal, alpha).reshape(-1)


    scores_low = y_hat_cal_CQR["q %1.2f" % alpha] - y_cal
    scores_high = y_cal - y_hat_cal_CQR["q %1.2f" % (1-alpha)] 
    scores_CQR = np.maximum(scores_low, scores_high)
    q_CQR = h.compute_quantile(scores_CQR, alpha)
    print(alpha)
    print(h.compute_coverage_len(y_test, np.maximum(0,y_hat_test - q), y_hat_test + q))
    print(h.compute_coverage_len(y_test, np.maximum(0,y_hat_test - q_normalized*sigma_test), y_hat_test + q_normalized*sigma_test))
    print(h.compute_coverage_len(y_test, np.maximum(0, y_hat_test  - q_array), y_hat_test  + q_array))
    print(h.compute_coverage_len(y_test, np.maximum(0, y_hat_test  - q_array_normalized*sigma_test), y_hat_test  + q_array_normalized*sigma_test))
    print(h.compute_coverage_len(y_test, np.maximum(0, y_hat_test_CQR["q %1.2f" % alpha] - q_CQR), y_hat_test_CQR["q %1.2f" % (1-alpha)]  + q_CQR))

0.1
(0.9382239382239382, 0.918918918918919, 58.78011321744518)
(0.9111969111969112, 0.8725868725868726, 49.50239916232735)
(0.9420849420849421, 0.9305019305019305, 60.66245336778787)
(0.9266409266409267, 0.8918918918918919, 50.99596635170149)
(0.918918918918919, 0.8996138996138996, 63.28153906901978)
0.15
(0.9227799227799228, 0.8725868725868726, 49.828786725592174)
(0.8841698841698842, 0.8108108108108109, 42.0913736009775)
(0.9227799227799228, 0.8764478764478765, 51.04677944164962)
(0.9034749034749034, 0.8455598455598455, 43.36336093364902)
(0.8957528957528957, 0.8455598455598455, 44.710219703082906)
0.2
(0.8957528957528957, 0.8378378378378378, 43.11205170901882)
(0.8571428571428571, 0.7374517374517374, 37.17751237689024)
(0.9073359073359073, 0.8494208494208494, 44.14483138378495)
(0.8725868725868726, 0.7606177606177607, 38.165004118170785)
(0.8532818532818532, 0.7799227799227799, 38.729309777153304)
0.25
(0.8687258687258688, 0.7722007722007722, 38.30273130622482)
(0.8416988416988417, 

In [29]:
q_array.shape

(100, 1)

In [54]:
a = q_array_normalized*sigma_test
a.shape

(100,)

(18404,)